# Left Atrium dataset preprocessing

In [ ]:
import sys
import os
import numpy as np
import cv2
import SimpleITK as sitk
import h5py

def load_nrrd(full_path_filename):
	data = sitk.ReadImage(full_path_filename)						
	data = sitk.Cast(sitk.RescaleIntensity(data),sitk.sitkUInt8)	
	data = sitk.GetArrayFromImage(data)								
	
	return(data)

def create_folder(full_path_filename):
	if not os.path.exists(full_path_filename):
		os.makedirs(full_path_filename)
	return

In [2]:
import os

n1 = 112						

N_train_patients = 100 	
N_test_patients  = 1		

train_files = os.listdir("2018_UTAH_MICCAI\\Training Set")
test_files = os.listdir("2018_UTAH_MICCAI\\Testing Set")


Image,Label = [],[]

In [5]:

for i in range(N_train_patients):
	patient_3DMRI_scan = load_nrrd(os.path.join("2018_UTAH_MICCAI\\Training Set",train_files[i],'lgemri.nrrd'))
	patient_3DMRI_cavity = load_nrrd(os.path.join("2018_UTAH_MICCAI\\Training Set",train_files[i],'laendo.nrrd'))//255
	patient_3DMRI_scan   = np.rollaxis(patient_3DMRI_scan,0,3)
	patient_3DMRI_cavity = np.rollaxis(patient_3DMRI_cavity,0,3)
	midpoint = patient_3DMRI_cavity.shape[0]//2
	n11, n12 = midpoint - int(n1/2), midpoint + int(n1/2)

	for n_slice in range(patient_3DMRI_scan.shape[2]):
		Image.append(patient_3DMRI_scan[n11:n12,n11:n12,n_slice])
		Label.append(patient_3DMRI_cavity[n11:n12,n11:n12,n_slice])

In [10]:
Image,Label = np.array(Image),np.array(Label)

temp = np.empty(shape=[Label.shape[0],n1,n1,2])
temp[:,:,:,0] = 1-Label
temp[:,:,:,1] = Label

Image,Label = np.reshape(Image,newshape=[-1,n1,n1,1]),np.reshape(temp,newshape=[-1,n1,n1,2])

train_mean,train_sd = np.mean(Image),np.std(Image)

Image = (Image-train_mean)/train_sd

for i in range(N_train_patients):
	Images,Labels = [],[]
	filen = os.path.join("2018_UTAH_MICCAI\\Training Set",train_files[i],'mri_norm2.h5')
	patient_3DMRI_scan = load_nrrd(os.path.join("2018_UTAH_MICCAI\\Training Set",train_files[i],'lgemri.nrrd'))
	patient_3DMRI_cavity = load_nrrd(os.path.join("2018_UTAH_MICCAI\\Training Set",train_files[i],'laendo.nrrd'))//255
	patient_3DMRI_scan   = np.rollaxis(patient_3DMRI_scan,0,3)
	patient_3DMRI_cavity = np.rollaxis(patient_3DMRI_cavity,0,3)
	midpoint = patient_3DMRI_cavity.shape[0]//2
	n11, n12 = midpoint - int(n1/2), midpoint + int(n1/2)
	for n_slice in range(patient_3DMRI_scan.shape[2]):

		Images.append(patient_3DMRI_scan[n11:n12,n11:n12,n_slice])
		Labels.append(patient_3DMRI_cavity[n11:n12,n11:n12,n_slice])
		
	h5f = h5py.File(filen,'w')
	h5f.create_dataset("image",			data=Images)
	h5f.create_dataset("label",			data=Labels)
	h5f.create_dataset("train.mean",	data=train_mean)
	h5f.create_dataset("train.sd",		data=train_sd)
	h5f.close()

# Data split preparation

In [4]:
import os
import random
all_files = os.listdir("LA")

In [3]:
random.shuffle(all_files)

In [8]:
train_set = all_files[:60]
val_set = all_files[60:80]
test_set = all_files[80:]

In [7]:
files = []
for filename in all_files:
    filename = filename[:-3]
    files.append(filename)

In [8]:
files = files[:-3]

In [11]:

with open("LA\\train.txt", "w") as f:
    for name in train_set:
        f.write(name[:-3] + "\n")

with open("LA\\val.txt", "w") as f:
    for name in val_set:
        f.write(name[:-3] + "\n")

with open("LA\\test.txt", "w") as f:
    for name in test_set:
        f.write(name[:-3] + "\n")